# Example of using Structured Streaming Output Mode
<font color='steelblue'>
<h3>
<span style="font-family:Arial; font-size:1.4em;">
Streaming data is coming via a network socket.<br>
<br>
To start transmission on network socket: <br>
On windows: ncat -kl 7777<br>
On MAC/Linux: nc -kl 7777

This streaming data is appended to a dataframe and is processed.<br>
 </span>
</h3>
</font>

<font color='gray'>
<span style="font-family:Arial; font-size:1.2em;">
There are 3 output modes that can be selected:
    <ul>
        <li><strong>Complete Mode:</strong> The entire updated result table is written to external storage. It is up to the storage connector to decide how to handle the writing of the entire table</li>
        <li><strong>Append Mode:</strong> Only new rows appended in the result table since the last trigger are written to external storage. This is applicable only for the queries where existing rows in the Result Table are not expected to change</li>
        <li><strong> Update Mode:</strong>Only the rows that were updated in the result table since the last trigger are written to external storage. This is different from Complete Mode in that Update Mode outputs only the rows that have changed since the last trigger. If the query doesn’t contain aggregations, it is equivalent to Append mode</li>
    </ul>
</span>
</font>

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *

In [ ]:
spark = SparkSession\
        .builder\
        .appName("Streaming Output modes")\
        .getOrCreate()

In [ ]:
 spark.sparkContext.setLogLevel("ERROR")

## Set up the host and port to listen to.
<br>
<font color='tomato'>
<b>
For testing: Change the option to complete, update and append
</b>
</font>

In [ ]:
host = 'localhost'
port = 7777
option = 'complete'

### Create DataFrame representing the stream of input lines from connection to host:port
### We're reading from the socket on the port where netcat is listening

In [ ]:
lines = spark\
        .readStream\
        .format('socket')\
        .option('host', host)\
        .option('port', port)\
        .load()

<h3>
<ol>
    <li>Split the lines into words</li>
    <li>Explode turns each item in an array into a separate row</li>
    <li>Alias sets the name of the column for the words</li>
    <li>The result - each word of input is a row in a table with one column named "word"</li>
</ol>
</h3>

In [ ]:
words = lines.select(explode(split(lines.value, ' ')).alias('word'))

In [ ]:
# Generate running word count
wordCounts = words.groupBy('word').count()

In [ ]:
if option == 'complete':
    query = wordCounts.writeStream\
                      .outputMode('complete')\
                      .trigger(processingTime="1 seconds")\
                      .format('console')\
                      .start()
elif option == 'update':
    query = wordCounts.writeStream\
                      .outputMode('update')\
                      .trigger(processingTime="1 seconds")\
                      .format('console')\
                      .start()
elif option == 'append':
    query = words.writeStream\
                 .outputMode('append')\
                 .trigger(processingTime="1 seconds")\
                 .format('console')\
                 .start()

In [ ]:
query.awaitTermination()